In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

A função buscar_html acessa o conteúdo do site. Ela cria uma requisição HTTP para conexão com o servidor e transforma o conteúdo HTML bruto em um formato legível para o BeautifulSoup.

In [1]:
url = 'http://books.toscrape.com/'

def buscar_html(url):
   
   try:
       resposta = requests.get(url)
       resposta.raise_for_status() 
       return BeautifulSoup(resposta.content, 'html.parser')
   except requests.exceptions.RequestException as e:
       print(f"Erro ao acessar {url}: {e}")
       return None

A função extrair_dados_livro analisa o HTML de cada livro, utiliza métodos do BeautifulSoup para encontrar as informações desejadas com base na estrutura de tags e classes CSS, e retorna esses dados de forma organizada para serem utilizados posteriormente no script.

In [3]:
def extrair_dados_livro(livro_soup):
    """Extrai título, preço, disponibilidade e classificação do livro."""
    titulo = livro_soup.h3.a['title']
    preco = livro_soup.find('p', class_='price_color').text
    disponibilidade = livro_soup.find('p', class_='instock availability').text.strip()
    classificacao = livro_soup.find('p', class_='star-rating')['class'][1]
    return titulo, preco, disponibilidade, classificacao

Loop Para Armazenar os dados dos livros em uma Lista e depois adicionar a um. no final do script é criado um DF.

In [4]:
if __name__ == '__main__':
    soup = buscar_html(url)
    
    lst_livros = []  # Lista para armazenar os livros

    if soup:
        livros = soup.find_all('article', class_='product_pod')
        
        for livro in livros:
            titulo, preco, disponibilidade, classificacao = extrair_dados_livro(livro)
            
            # Dicionario com os dados do livro
            livro_data = {
                'Título': titulo, 
                'Preço': preco, 
                'Disponibilidade': disponibilidade, 
                'Classificação': classificacao
            }

            lst_livros.append(livro_data) # Adiciona o dicionário à lista

    # Criação do DF
    df = pd.DataFrame(lst_livros)
   

Transformação basica dos dados

In [5]:
df.head()

,Título,Preço,Disponibilidade,Classificação
0,A Light in the Attic,£51.77,In stock,Three
1,Tipping the Velvet,£53.74,In stock,One
2,Soumission,£50.10,In stock,One
3,Sharp Objects,£47.82,In stock,Four
4,Sapiens: A Brief History of Humankind,£54.23,In stock,Five


In [37]:
df.describe()

,Título,Preço,Disponibilidade,Classificação
count,20,20,20,20
unique,20,20,1,5
top,A Light in the Attic,£51.77,In stock,One
freq,1,1,20,6


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Título           20 non-null     object
 1   Preço            20 non-null     object
 2   Disponibilidade  20 non-null     object
 3   Classificação    20 non-null     object
dtypes: object(4)
memory usage: 772.0+ bytes


In [39]:

#Convertendo os preços para float
df['Preço'] = df['Preço'].str.replace('£', '', regex=False).astype(float)

#criando uma coluna com numeros.
class_map= {
    'One': 1,
    'Two': 2,
    'Three': 3,
    'Four': 4,
    'Five': 5
}

df['class_num'] = df['Classificação'].map(class_map)

In [41]:
df.describe(include='all')

,Título,Preço,Disponibilidade,Classificação,class_num
count,20,20.000000,20,20,20.000000
unique,20,NaN,1,5,NaN
top,A Light in the Attic,NaN,In stock,One,NaN
freq,1,NaN,20,6,NaN
mean,NaN,38.048500,NaN,NaN,2.850000
std,NaN,15.135231,NaN,NaN,1.565248
min,NaN,13.990000,NaN,NaN,1.000000
25%,NaN,22.637500,NaN,NaN,1.000000
50%,NaN,41.380000,NaN,NaN,3.000000
75%,NaN,51.865000,NaN,NaN,4.000000


In [42]:
df.to_csv('dados_livros.csv', index=False, encoding='utf-8')